In [1]:
import pandas as analytics
import numpy as maths
from math import exp
from numba import cuda , jit
import sympy
import warnings 
warnings.filterwarnings("ignore")

In [2]:
df_data = analytics.read_csv('data1.csv',header = None , names = ['x0','x1','y'])
df_data

,x0,x1,y
0,22.930,-20.9020,1
1,12.410,2.0033,1
2,-20.686,-33.3030,1
3,46.974,-13.5550,1
4,41.965,20.3680,1
...,...,...,...
195,-29.232,28.4410,-1
196,-10.159,7.3065,-1
197,-32.026,14.9380,-1
198,-49.533,-11.1280,-1


In [11]:
class neural_network :
    
    def __init__(self,df_data , number_of_hidden_layers = 1, number_of_neurons = [2,1], epsilon = 5e-2, alpha = 0.2 , hidden_function = 'relu', output_function = 'sigmoid') :
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_neurons = number_of_neurons
        self.df_data = df_data
        self.epsilon = epsilon 
        self.N = df_data.shape[0]
        self.alpha = alpha
        functions_list = {'relu' : self.relu , 'sigmoid' : self.sigmoid }
        self.hidden = functions_list[hidden_function]
        self.output = functions_list[output_function]

    def relu(self,x) :
        """Hidden Activation Function"""
        return max(0,x)
    
    def sigmoid(self,x):
        """Output Function"""
        return 1 / (1 + exp(-x))


    def calculate_derivative(self,x,func):
        if func == self.sigmoid :
            derivative = func(x) * ( 1- func(x))
        if func == self.relu :
            if x > 0 : x = 1
            derivative = func(x)

        return derivative


    
    def initialisation(self):
        weights = []
        bias = []
        for l in range(self.number_of_hidden_layers + 1):
            if l == 0 : previous_layer = df_data.shape[1] - 1
            else : previous_layer = self.number_of_neurons[l-1]
            present_layer = self.number_of_neurons[l]
            weights.append(maths.random.random((previous_layer,present_layer)))
            bias.append(maths.random.random(present_layer).reshape(-1,1))
        self.weights = weights
        self.bias = bias


    def forward_propagation(self,x):
        activations = []
        activations.append(x)   # input data point is the initial activation
        
        hidden_layers = []
        
        for l in range(self.number_of_hidden_layers):
            neurons = self.weights[l] @ activations[l] + self.bias[l]   # it actually should be activations[l+1] but because indexing in python starts from 0, so it is activations[l]
            activation = maths.matrix([self.hidden(float(neuron)) for neuron in neurons]).reshape(-1,1)
            hidden_layers.append(neurons)
            activations.append(activation)
        data = self.weights[-1].T @ activations[-1] + self.bias[-1]
        activations.append(self.output(data))
        
        self.hidden_layers = hidden_layers
        self.activations = activations


    def backward_propagation(self) :
      
        deltas = []
        data = self.weights[-1].T @ self.activations[-2]  + self.bias[-1]
        delta = maths.matrix(maths.multiply(self.activations[-1],  self.calculate_derivative(data, self.output))).reshape(-1,1)
        deltas.append(delta)
        
        grad_weights = []
        grad_biases = []
        for l in range(self.number_of_hidden_layers , -1, -1 ): # it should be -1 only. Some errors in activations is not letting it run.
            grad_weight = deltas[0] @ self.activations[l].T
            grad_bias = deltas[0]
            delta = maths.multiply((deltas[0] @ self.weights[1].T).T , maths.matrix([self.calculate_derivative(float(i),self.hidden) for i in self.hidden_layers[l-1]]).reshape(-1,1))
            deltas.insert(0,delta)
            grad_weights.append(grad_weight)
            grad_biases.append(grad_bias)
    
        return grad_weights, grad_biases


    def has_converged(self, prev_weights, prev_bias) :
        self._converged = (maths.linalg.norm(maths.matrix(self.weights[0]) - maths.matrix(prev_weights[0])) < self.epsilon) and (maths.linalg.norm(maths.matrix(self.bias[0]) - maths.matrix(prev_bias[0])) < self.epsilon)


    def update_weights(self):
        
        prev_weights = [w + 1 for w in self.weights]
        prev_bias = [b + 1 for b in self.bias]
        
        self._converged = False
        
        while not self._converged:
            prev_weights = self.weights.copy()
            prev_bias = self.bias.copy()
            
            for m in range(self.N) :
                x = maths.matrix(self.df_data.iloc[m][:-1]).reshape(-1,1)
                y = self.df_data.iloc[m][-1]
                self.forward_propagation(x)
                grad_weights, grad_bias = self.backward_propagation()
                for l in range(self.number_of_hidden_layers) :
                    self.weights[l] = self.weights[l] - self.alpha/self.N * sum(grad_weights)
                    self.bias[l] = self.bias[l] - self.alpha/self.N * sum(grad_bias)
                    
            
            self.has_converged(prev_weights, prev_bias)

In [15]:
number_of_hidden_layers = 1
number_of_neurons = [2,1]
epsilon = 5e-2
alpha = 0.2
hidden_function = 'relu' 
output_function = 'sigmoid'


ann = neural_network(df_data , 
                     number_of_hidden_layers = number_of_hidden_layers ,
                     number_of_neurons = number_of_neurons ,
                     epsilon = epsilon ,
                     alpha = alpha ,
                     hidden_function = hidden_function ,
                     output_function = output_function 
                    )
ann.initialisation()
ann.update_weights()

In [16]:
ann.weights

[matrix([[0.78718858, 0.50823176],
         [0.36402013, 0.39135609]]),
 array([[0.89502841],
        [0.27438637]])]

In [17]:
ann.bias

[matrix([[0.69192556],
         [0.61308727]]),
 array([[0.4916638]])]